In [1]:
import networkx as nx 
import random
import matplotlib.pyplot as plt
import numpy as np 
import math 
import copy 
import graphviz
from scipy.optimize import fsolve
from typing import List
import sympy as sp
from sympy import Symbol
from collections import deque
import pickle
import lattpy as lp
import import_ipynb
from arrival_networkx import *
from iteration import *

importing Jupyter notebook from arrival_networkx.ipynb
importing Jupyter notebook from iteration.ipynb


In [2]:
def inner_algorithm(L, s, f):
    # L is the current sub-instance, represented by lower (a) and upper (b) bounds.
    # s is the principle slice (∗, ∗, s3).
    # f is the function being analyzed.
    print("Inner algorithm called with L =", L, "and s =", s, ".")
    x, y = L
    
    a = x.copy()
    b = y.copy()
    fixed_index = None
    for i in range(len(s)):
        if not np.isnan(s[i]):
            fixed_index = i
            a[i] = s[i]
            b[i] = s[i]
            break
    free_idx_1 = (fixed_index + 1) % 3
    free_idx_2 = (fixed_index + 2) % 3    
    
    def is_up_set_witness(p1, p2):
        f_p1 = f(p1)
        f_p2 = f(p2)
        return p1[0] == p2[0] and p1[1] >= p2[1] and p1[1] >= f_p1[1] and f_p2[1] >= p2[1] and p1[2] >= f_p1[2] and p2[2] >= f_p2[2]
    
    def is_down_set_witness(p1, p2):
        f_p1 = f(p1)
        f_p2 = f(p2)
        return p1[0] == p2[0] and p1[1] <= p2[1] and p1[1] <= f_p1[1] and f_p2[1] <= p2[1] and p1[2] <= f_p1[2] and p2[2] <= f_p2[2]
    

    mid = (a + b) // 2
    
    bot = np.full(len(mid), np.nan)
    bot[fixed_index] = a[fixed_index]
    bot[free_idx_1] = np.floor((a[free_idx_1] + b[free_idx_1])/2)
    bot[free_idx_2] = a[free_idx_2]
    
    
    top = np.full(len(mid), np.nan)
    top[fixed_index] = b[fixed_index]
    top[free_idx_1] = np.floor((a[free_idx_1] + b[free_idx_1])/2)
    top[free_idx_2] = b[free_idx_2]

    left = np.full(len(mid), np.nan)
    left[fixed_index] = a[fixed_index]
    left[free_idx_1] = a[free_idx_1]
    left[free_idx_2] = np.floor((a[free_idx_2] + b[free_idx_2])/2)
    
    right = np.full(len(mid), np.nan)
    right[fixed_index] = b[fixed_index]
    right[free_idx_1] = b[free_idx_1]
    right[free_idx_2] = np.floor((a[free_idx_2] + b[free_idx_2])/2)
    
    ##############################################################################################################
    ################# STEP 1 #####################################################################################
    # if is_up_set_witness(a, bot):
    #     # mid belongs to Up(f_s)
    #     print(a , bot)
    #     print("up set witness")
    #     raise Exception("up set witness")
        
    # elif is_down_set_witness(top, b):
    #     # mid belongs to Down(f_s)
    #     # return a, mid
    #     print(top, b)
    #     print("down set witness")
    #     raise Exception("down set witness")
        
    # elif is_up_set_witness(a, left):
    #     # return left, b
    #     print(a, left)
    #     print("up set witness")
    #     raise Exception("up set witness")
    
    # elif is_down_set_witness(right, b):
    #     # return a, right
    #     print(right, b)
    #     print("down set witness")
    #     raise Exception("down set witness")
    
    
    ##############################################################################################################
    ################# STEP 2 #####################################################################################
    f_mid = f(mid)
    if mid[free_idx_1] <= f_mid[free_idx_1] and mid[free_idx_2] <= f_mid[free_idx_2]:
        # mid belongs to Up(f_s)
        return mid, b
        
    elif mid[free_idx_1] >= f_mid[free_idx_1] and mid[free_idx_2] >= f_mid[free_idx_2]:
        # mid belongs to Down(f_s)
        return a, mid
        
    elif mid[free_idx_1] <= f_mid[free_idx_1] and mid[free_idx_2] > f_mid[free_idx_2]:
        if mid[fixed_index] <= f_mid[fixed_index]:
            # step 2, case 3a
            f_right = f(right)
            if right[fixed_index] > f_right[fixed_index]:
                return None ## order preserving violation
            elif right[free_idx_1] < f_right[free_idx_1]:
                ## case 3a ii lemma 12
                p =right 
                pass
            else:
                return a, right 
        else:
            # step 2, case 3b
            f_bot = f(bot)
            if bot[fixed_index] > f_bot[fixed_index]:
                print("order preserving violation, bot: ", bot,"f_bot:", f_bot, "mid:", mid, "f_mid:", f_mid)
                return None ## order preserving violation
            elif bot[free_idx_2] > f_bot[free_idx_2]:
                ### case 3b ii lemma 12
                p = bot
                pass
            else:
                return bot, b
            
    elif mid[free_idx_1] > f_mid[free_idx_1] and mid[free_idx_2] <= f_mid[free_idx_2]:
        if mid[fixed_index] <= f_mid[fixed_index]:
            # step 2, case 3a
            f_top = f(top)
            if top[fixed_index] > f_top[fixed_index]:
                return None ## order preserving violation
            elif top[free_idx_2] < f_top[free_idx_2]:
                ## case 3a ii lemma 12
                p = top
                print("incomplete case")
                pass
            else:
                return a, top
        else:
            # step 2, case 3b
            f_left = f(left)
            if left[fixed_index] > f_left[fixed_index]:
                print("ov2")
                return None ## order preserving violation
            elif left[free_idx_1] > f_left[free_idx_1]:
                ### case 3b ii lemma 12
                p = left
                print("incomplete case")
                pass
            else:
                return left, b


    # Terminal phase: Check if the instance size is reduced sufficiently.
    if np.all(b - a <= 1):
        print("Terminal phase reached.")

    # Return the final sub-instance containing a solution, or indicate a violation was found.
    return a, b

In [3]:
def outer_algorithm(f, x, y):
    
    # Initialize the search space
    k = len(x)  # Dimensionality of the problem
    print("Initial search space: x =", x, "y =", y)

    # Iteratively refine the search space
    while np.any(y - x >= 2):
        # Determine the dimension i with the largest gap between x and y
        i = np.argmax(y - x)
        
        # Create a principle slice s centered between x and y in the selected dimension
        s = np.full(k, np.nan, dtype=float)
        s[i] = x[i] + (y[i] - x[i]) // 2

        # Invoke the inner algorithm on the current sub-instance and principle slice
        La_b = (x.copy(), y.copy())
        result = inner_algorithm(La_b, s, f)
        
        
        # Check the result of the inner algorithm
        if result is None:
            print("An order preservation violation was found.")
            return None
        
        
        # Update x and y to the refined sub-instance bounds
        x, y = result
        
        print("Refined bounds: x =", x, "y =", y)

    return x, y



## Decomposition of n-dimensional instance

In [4]:
class AlgorithmL():
    def __init__(self, f,Lattice):
        self.lower, self.upper = Lattice
        self.f = f
        self.P = set()    
        
    def evaluation_function_A(self, x):
        l,u = self.construct_l_u(x)
        algob = AlgorithmB(x, self.f, (l,u))
        y = algob.solve()
        self.P.add(np.concatenate([x,y]))
        
        return self.f(np.concatenate([x,y]))[:len(x)]
        
    def construct_l_u(self, x):
        D = [p[len(x):] for p in self.P if np.all(p[:len(x)] <= x)]
        l = self.lower[len(x):] if not D else np.max(np.array(D), axis=0)
        
        U = [p[len(x):] for p in self.P if np.all(p[:len(x)] >= x)]
        u = self.upper[len(x):] if not U else np.min(np.array(U), axis=0)
        
        return l,u
        

    def solve(self):
        algoA = AlgorithmA(self.evaluation_function_A, (self.lower[:3],self.upper[:3]))
        return algoA.solve_tarski_3d()


class AlgorithmA():
    def __init__(self,  sub_function, Lattice_A):
        self.lower, self.upper = Lattice_A
        self.sub_function = sub_function 
        
        
    def solve_tarski_3d(self):
        final_sub_instance = outer_algorithm(self.sub_function, self.lower, self.upper)
        print("Outcome:", final_sub_instance)
        srt , end = final_sub_instance
        
        ## loop through all the points between srt and end inclusive
        for i in range(int(srt[0]),int(end[0])+1):
            for j in range(int(srt[1]),int(end[1])+1):
                for k in range(int(srt[2]),int(end[2])+1):
                    if np.all(self.sub_function(np.array([i,j,k])) == np.array([i,j,k])):
                        return np.array([i,j,k])
        
        return None


class AlgorithmB():
    def __init__(self, x, f , Lattice_B):
        self.lower, self.upper = Lattice_B
        self.x = x
        self.f = f
        
    def solve(self):
        if len(self.lower) > 3:
            algoL = AlgorithmL(self.sub_function,(self.lower,self.upper))
            return algoL.solve()
        elif len(self.lower) == 3:
            algoA = AlgorithmA(self.sub_function, (self.lower,self.upper))
            return algoA.solve_tarski_3d()
    
    def sub_function(self,y):
        return self.f(np.concatenate([self.x,y]))[- len(y):]